In [21]:
import numpy as np
import pandas as pd

In [22]:
# Load the CSV file into a DataFrame
csv_file_path = './content/BTC_2019_2023_2h.csv'  # Replace with your CSV file path
df = pd.read_csv(csv_file_path)

# Add the 'signals' column with all entries set to 0
df['signals'] = 0

# Add the 'trade_type' column with all entries set to NaN
df['trade_type'] = np.nan

# Save the updated DataFrame back to the CSV file (optional)
df.to_csv(csv_file_path, index=False)

# Display the first few rows to verify
print(df.head())


   Unnamed: 0             datetime      open      high       low     close  \
0           0  2019-09-08 16:00:00  10000.00  10000.00  10000.00  10000.00   
1           1  2019-09-08 18:00:00  10344.77  10357.53  10337.43  10340.12   
2           2  2019-09-08 20:00:00  10340.12  10391.90  10324.77  10391.90   
3           3  2019-09-08 22:00:00  10392.59  10412.65  10366.57  10391.63   
4           4  2019-09-09 00:00:00  10391.63  10391.63  10391.63  10391.63   

     volume  signals  trade_type  
0     0.002        0         NaN  
1   471.659        0         NaN  
2  1273.030        0         NaN  
3  1351.600        0         NaN  
4     0.000        0         NaN  


In [23]:

# Calculate EMAs
df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
df['ema_40'] = df['close'].ewm(span=40, adjust=False).mean()

# Generate signals
def generate_signals(df):
    signals = []
    for i in range(len(df)):
        if i == 0:
            signals.append(0)
        elif df['ema_20'][i] > df['ema_40'][i] and df['ema_20'][i-1] <= df['ema_40'][i-1]:
            signals.append(1)
        elif df['ema_20'][i] < df['ema_40'][i] and df['ema_20'][i-1] >= df['ema_40'][i-1]:
            signals.append(-1)
        else:
            signals.append(0)
    return signals

df['signals'] = generate_signals(df)

# Determine trade types based on signals
def determine_trade_type(row):
    if row['signals'] == 1 and (pd.isna(row['trade_type']) or row['trade_type'] == 'close'):
        return 'long'
    elif row['signals'] == -1 and (pd.isna(row['trade_type']) or row['trade_type'] == 'close'):
        return 'short'
    elif row['signals'] == -1 and row['trade_type'] == 'long':
        return 'close'
    elif row['signals'] == 1 and row['trade_type'] == 'short':
        return 'close'
    elif row['signals'] == -2:
        return 'short'
    elif row['signals'] == 2:
        return 'long'
    else:
        return row['trade_type']

df['trade_type'] = df.apply(determine_trade_type, axis=1)

# Replace NaN values in trade_type
df['trade_type'] = df['trade_type'].fillna('hold')


# Display the DataFrame with trade types
print(df)


       Unnamed: 0             datetime      open      high       low  \
0               0  2019-09-08 16:00:00  10000.00  10000.00  10000.00   
1               1  2019-09-08 18:00:00  10344.77  10357.53  10337.43   
2               2  2019-09-08 20:00:00  10340.12  10391.90  10324.77   
3               3  2019-09-08 22:00:00  10392.59  10412.65  10366.57   
4               4  2019-09-09 00:00:00  10391.63  10391.63  10391.63   
...           ...                  ...       ...       ...       ...   
18900       18900  2023-12-31 16:00:00  42489.60  42741.60  42465.40   
18901       18901  2023-12-31 18:00:00  42651.90  42766.70  42570.10   
18902       18902  2023-12-31 20:00:00  42659.90  42724.50  42543.30   
18903       18903  2023-12-31 22:00:00  42559.00  42629.50  42083.10   
18904       18904  2024-01-01 00:00:00  42314.00  42832.00  42289.60   

          close     volume  signals trade_type        ema_20        ema_40  
0      10000.00      0.002        0       hold  10000.0000

In [24]:
df.to_csv ('try2.csv', index=False)